# Lecture on the Wigner's cats

    by M.Süzen
    (c) 2025

Prerequisite to this tutorial is finishing the `wigner_semicircle.ipynb` tutorial first for a background. 

In this lecture we will generate a mixed ensemble. A mixed ensemble implies different order matrices at a given `degree of mixture` (DoM). DoMs are a principled way to generate those different order matrices via `Mixed Matrix Ensemble Sampling (MMES) algoritm`. [suzen21]. 

## Load Leymosun components

We import the following functionality from `leymosun` package. 

`goe` : Gaussian Orthogonal Matrix generator.   
`mixed_ensemble`: Ensemble generator, for mixed ensemble.  
`empirical_spectral_density`: Compute eigenvalues and their density $\rho(\lambda)$.     
`wigner`: Compute the Wigner density.

See each method's documentation for more details. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from leymosun.gaussian import goe, wigner
from leymosun.matrix import mixed_ensemble 
from leymosun.spectral import empirical_spectral_density
from leymosun.stats import bootstrap_observed_matrix_ci
import leymosun
leymosun.__version__

## Generating mixed matrix ensembles

A mixed matrix ensemble is formed by choosing different sized matrices. Recent work on this demonstrated [Suzen21] a principled way of generating a mixed ensemble. After setting a reference $N$ matrix order, generating mixed-sizes from a Binomial distribution with re-interpretation of the probability of success as the degree of mixture. This mixture defines which new matrix orders to generate and how many.  

`Mixed Matrix Ensemble Sampling (MMES)`: The idea is to get a new matrix order $N_{i}$ at the reference matrix order $N_{ref}$ via binomimal distribution at the given probability-so called the Degree-of-Mixture (DoM) $\mu$: $N_{i} \sim Bin(N_{ref}, \mu)$. Repeating this procedure with the ensemble size $M$ will generate the mixed ensembles matrix-orders we should generate. `mixed_ensemble` method from `leymosun` exactly do this.  

In [ ]:
matrix_order = 1000
ensemble_size = 100
degree_of_mixture = 0.8
ensemble_sample = mixed_ensemble(matrix_order, ensemble_size, degree_of_mixture, goe)

**Exercise 1** Inspect ensemble sample and identify how matrix-orders are distributed.   
Hint: Recall Central Limit Theorem.  

## Wigner's Cats

Now, we are in position to generate so called "Wigner's Cats". It isn't only a playful reference to Wigner's mathematical treatment to quantum mechanics but the resulting shapes of emprical eigenvalue distributions look like cats for given degree-of-mixture. 

### Peridodic Boundary Conditions on the spectra

An important detail here is, when we analyze spectra of different size matrices, we need to align their spectra to reference length, i.e., reference matrix order $N_{ref}$. The alignment is achived via replicating the spectra up to the reference order. This is also called periodic boundary conditions or cyclic sequence up to an order. We apply this cyclic order after computing the spectra of each mixed ensemble member. This procedure results in all equal lenght spectra. Analysis of density is then applied. `emprical_spectral_density` method handles this via its `mixed_order` parameter, pooling all spectra with the mixed ensemble, i.e., GOE here.

In [ ]:
eigenvalues, densities, locations = empirical_spectral_density(
    ensemble_sample, mmes_order=matrix_order, scale="wigner"
)

We want to report 95% confidence intervals. 

In [ ]:
observed_mean, observed_upper, observed_lower = bootstrap_observed_matrix_ci(densities)

Standard analytical computation remains the same with the `wigner` method. 

In [ ]:
dwigner = wigner(locations, domain_boundary=2.0)

Now plotting the densities, we observer the `Wigner's Cat` (`M-shaped`) desity at the degree of mixture $\mu=0.8$.

In [ ]:
yerr = np.array([observed_mean-observed_lower, observed_upper-observed_mean])
plt.bar(locations, observed_mean, width=0.02, alpha=0.3, align='center') 
plt.errorbar(locations, observed_mean, yerr=yerr, fmt=' ', capsize=5)  
plt.title("Wigner's Semicircle law with Leymosun \n with Wigner's cat (M-shaped) density at $\mu=0.8$ ")
plt.xlabel("Eigenvalue Locations")
plt.ylabel("Density")

## Conclusion

We have demonstrated how the mixed ensembles can be formed and the resulting empirical spectral distribution deviates from the semi-circle law. 

**Exercise 2** Run the numerical experiments at very high degree-of-mixture $\mu$ parameters, such as $0.97$. What did you observe on the emprical spectral density?

## Reference

[Suzen21] Empirical deviations of semicircle law in mixed-matrix ensembles, M. Suzen
hal-03464130 (2021) [url](https://hal.science/hal-03464130)    
`Introduction of mixed matrix ensembles and MMES algorithm with M-shaped (Wigner's Cat) density.`